# Distance Matrix Job

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')

from domain_context import DomainContext

### Setup

In [3]:
TEMP_PATH = '../../temp'
HOST      = 'http://localhost:8000'
API_TOKEN = 'e3ff025094fe0ee474501bbeda0a2a44e80230c1'

ctx = DomainContext(API_TOKEN, HOST, TEMP_PATH)

In [4]:
# ctx.api.similarity_cells_versions()
# ctx.api.remove_similarity_cells_by_version(18)

### Execute jobs

In [5]:
ctx.svd_distance_matrix_job.execute()

2022-12-21 20:32:28,728 :: INFO :: SurpriseDistanceMatrixJob :: Start
2022-12-21 20:32:30,975 :: INFO :: InteractionRepository :: Page 1/2 - Interactions 50000/96605
2022-12-21 20:32:33,184 :: INFO :: InteractionRepository :: Page 2/2 - Interactions 96605/96605
2022-12-21 20:32:33,184 :: INFO :: InteractionRepository :: 96605 Total Interactions 
2022-12-21 20:32:33,363 :: INFO :: InteractionService :: Filter by rating scale: [1, 2, 3, 4, 5]
2022-12-21 20:32:33,370 :: INFO :: InteractionService :: Filtered: 100.0%
2022-12-21 20:32:33,370 :: INFO :: InteractionService :: Filter interactions by user_n_interactions >= 20
2022-12-21 20:32:33,375 :: INFO :: InteractionService :: Filtered interactions: 95.2%
2022-12-21 20:32:33,375 :: INFO :: InteractionService :: Excluded interactions: 4682
2022-12-21 20:32:46,403 :: INFO :: InteractionService :: Unrated interactions: 97.4%
2022-12-21 20:32:46,491 :: INFO :: RatingMatrixService :: Train interactions: 91923 - Users: 1098, Items: 3139
2022-12-

In [6]:
ctx.nmf_distance_matrix_job.execute()

2022-12-21 20:39:17,069 :: INFO :: SurpriseDistanceMatrixJob :: Start
2022-12-21 20:39:19,447 :: INFO :: InteractionRepository :: Page 1/2 - Interactions 50000/96605
2022-12-21 20:39:21,785 :: INFO :: InteractionRepository :: Page 2/2 - Interactions 96605/96605
2022-12-21 20:39:21,786 :: INFO :: InteractionRepository :: 96605 Total Interactions 
2022-12-21 20:39:21,990 :: INFO :: InteractionService :: Filter by rating scale: [1, 2, 3, 4, 5]
2022-12-21 20:39:21,995 :: INFO :: InteractionService :: Filtered: 100.0%
2022-12-21 20:39:21,996 :: INFO :: InteractionService :: Filter interactions by user_n_interactions >= 20
2022-12-21 20:39:22,001 :: INFO :: InteractionService :: Filtered interactions: 95.2%
2022-12-21 20:39:22,001 :: INFO :: InteractionService :: Excluded interactions: 4682
2022-12-21 20:39:35,396 :: INFO :: InteractionService :: Unrated interactions: 97.4%
2022-12-21 20:39:35,491 :: INFO :: RatingMatrixService :: Train interactions: 91923 - Users: 1098, Items: 3139
2022-12-